In [2]:
import ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
import pandas as pd
import chromadb
from chromadb import EmbeddingFunction
import tqdm
import time
from IPython.display import Markdown
import fitz

In [3]:
def extract_text_from_pdf(pdf_path):
    try:
        # Open the PDF file
        with fitz.open(pdf_path) as pdf_document:
            # Initialize an empty string to store the extracted text
            text = ""
            # Iterate through each page in the PDF
            for page_number in range(len(pdf_document)):
                # Get the current page
                page = pdf_document.load_page(page_number)
                # Extract text from the current page
                page_text = page.get_text()
                # Append the extracted text to the result
                text += page_text
            # Return the extracted text
            return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return None

In [4]:
data = extract_text_from_pdf("D:/ML_Projects/Llama-RAG/T L SINGAL BOOK.pdf")

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 5000,    
    chunk_overlap = 100,
    length_function = len,
    add_start_index = True
)

In [6]:
texts = text_splitter.create_documents([data])
print(texts[0].page_content)

Wireless Communications
About the Author
T L Singal is currently Professor in the Department of Electronics and 
Communication Engineering, Chitkara Institute of Engineering and Techno-
logy, Rajpura, Punjab. He graduated in Electronics and Communi cation 
Engineering from National Institute of Technology (NIT), Kurukshetra, and 
post-graduated in Electronics and Communication Engineering from Punjab 
Technical University, Jalandhar. He has worked in the field of Wireless 
and Cellular Communications with leading telecom organisations in India 
and USA for about 20 years and has managed various VHF/UHF Wireless 
Communication Projects in reputed companies like HAL, Hyderabad, and 
PSIDC, Chandigarh. He has also visited Germany on business trips for tech-
nical know-how on the transfer of Multi-Access Wireless Communication 
Systems during 1990–92. In his last association with Flextronics International 
Inc., Dallas (Texas), USA, he held the position of Senior Network Consultant, 
offer

In [7]:
docs = []

for chunk in texts:
    docs.append(chunk.page_content)

print(docs[:2])

['Wireless Communications\nAbout the Author\nT L Singal is currently Professor in the Department of Electronics and \nCommunication Engineering, Chitkara Institute of Engineering and Techno-\nlogy, Rajpura, Punjab. He graduated in Electronics and Communi cation \nEngineering from National Institute of Technology (NIT), Kurukshetra, and \npost-graduated in Electronics and Communication Engineering from Punjab \nTechnical University, Jalandhar. He has worked in the field of Wireless \nand Cellular Communications with leading telecom organisations in India \nand USA for about 20 years and has managed various VHF/UHF Wireless \nCommunication Projects in reputed companies like HAL, Hyderabad, and \nPSIDC, Chandigarh. He has also visited Germany on business trips for tech-\nnical know-how on the transfer of Multi-Access Wireless Communication \nSystems during 1990–92. In his last association with Flextronics International \nInc., Dallas (Texas), USA, he held the position of Senior Network Co

In [8]:
from chromadb.utils import embedding_functions

In [9]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

d:\ML_Projects\Llama-RAG\parth\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from tqdm import tqdm

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        embeddings = sentence_transformer_ef([docs])
        return embeddings

In [11]:
def create_chroma_db(docs,name):
    chroma_client = chromadb.PersistentClient(path="D:/ML_Projects/Llama-RAG/database")
    db = chroma_client.get_or_create_collection(
        name=name, embedding_function=sentence_transformer_ef)
    
    initial_size = db.count()
    for i, d in tqdm(enumerate(docs), total=len(docs), desc="Creating Chroma DB"):
        db.add(
            documents=d,
            ids=str(i + initial_size)
        )
        time.sleep(0.5)
    return db


def get_chroma_db(name):
    chroma_client = chromadb.PersistentClient(path="D:/ML_Projects/Llama-RAG/database")
    return chroma_client.get_collection(name=name, function=EmbeddingFunction())

In [12]:
db = create_chroma_db(docs, "db_rag")
db.count()

Creating Chroma DB:   4%|▎         | 45/1201 [00:26<11:26,  1.68it/s]


KeyboardInterrupt: 

In [ ]:
def get_relevant_passages(query, db, n_results=5):
    passages = db.query(query_texts=[query], n_results=n_results)['documents'][0]
    return passages

In [ ]:
ques = "Is the information correct  "
passages = get_relevant_passages(ques, db, n_results=7)

Markdown(passages[0]+passages[1])

book and I thank the many people responsible for the publication of this book. To begin with, my thanks 
to Aman Taneja, for appreciating my willingness to write the script, and introducing me to the TMH team 
who took up this project. My sincere thanks are also due to Suman Sen and Manish Choudhary for their 
continuous encouragement to include the reviewers’ feedback and comments during the preparation of 
the text. I also want to thank Vibha Mahajan, P L Pandita, Sohini Mukherjee, Rachna Sehgal and other 
staff members of Tata McGraw Hill Education for their excellent coordination during copyediting and 
 production stages of the book.
The dream of my beloved parents, who wished me to be a mentor for aspiring young engineering students, 
is fulfilled through the publication of this book. Their blessings are similar to those bestowed by the Almighty. 
I thank my wife, Pinki; daughter, Ritu; and son, Pankaj for their inspiration, patience and continuous support.the subject.’
‘The script has given due importance to the propagation models and Antenna Design Considerations. 
Feedback
I hope every reader will find this book rich in content as well as pedagogical features. You will certainly enjoy 
the simplified yet extensive and elaborate approach to every new topic under discussion. I have tried my level 
best to make it a complete book on Wireless Communication but I believe there is always scope for improve-
ment in all our efforts. Your valuable suggestions, feedback, or any other comments are most welcome at my 
email id: tarsemsingal@gmail.com. I am sure this will enable me to enhance the content of the next edition.
T L Singal
Publisher’s Note
Do you have any feedback? We look forward to receive your views and suggestions for improvement. The 
same can be sent to tmh.ecefeedback@gmail.com mentioning the title and author’s name in the subject line.
Preface 
xvii
II 
List of Important Symbols
γ
path-loss exponent or distance-power 
gradient
λc

In [ ]:
def ollama_llm(question, context):
    formatted_prompt = f'''
    Question: {question}\n\nContext: {context}
    Answer the question in 1250-2500 words strictly based on the context given.If the question is asking for a code then also explain the algorithm.If the question contains equations, explain them. Incase of spelling errors in the context, make spelling corrections in the response.
    If the information is not sufficient then give output as "Info not good to answer".
    '''
    response = ollama.chat(model='mistral', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

In [ ]:
def list_to_string(passages):
    content = ""
    for passage in passages:
        content += passage + "\n"
    return content

In [ ]:
answer = ollama_llm(ques,list_to_string(passages))
Markdown(answer)

 Title: A Comprehensive Textbook on Wireless Communications: From Fundamentals to Advanced Technologies

Introduction:
Wireless Communications is an essential textbook designed for undergraduate and postgraduate students majoring in wireless communications, cellular and mobile communications, wireless communication systems and networks, and advanced communication systems. This comprehensive book covers the fundamental concepts of wireless communications and then delves deeper into cellular architecture, designs, and emerging wireless networking technologies. The text is organized to provide sufficient flexibility for designing courses based on the latest trends in the advancement of wireless and cellular communication technologies.

Chapter Overview:
The following sections provide an overview of each chapter's content, its relevance to various target courses, and a brief explanation of some essential concepts covered within.

1. Wireless Communications Fundamentals: This chapter introduces the reader to wireless communications, their importance, and the fundamental principles governing their operation. Topics include electromagnetic waves, propagation, radio frequency spectrum, modulation schemes, and multiple access methods. This chapter is a prerequisite for further study in wireless communications, cellular systems, and networking technologies.

Target Courses: Wireless/mobile communications, Cellular and mobile communications, Wireless communication systems and networks, and Advanced communication systems.

2. Principles of Wireless Communications: This chapter focuses on the basic principles of wireless communications, such as free-space propagation, diffraction and reflection, polarization, and multipath effects. It also covers radio wave propagation models like Hata, COST-231, and Longley-Rice. The applications of these models in network planning are discussed.

Target Courses: Wireless/mobile communications, Cellular and mobile communications.

3. Antennas for Wireless Communications: This chapter covers the design, types, and application of antennas used in wireless communications. Topics include omnidirectional, directional, Yagi-Uda, dipole, patch, and microstrip antennas. The relationship between gain, radiation efficiency, and directivity is also discussed.

Target Courses: Wireless/mobile communications, Cellular and mobile communications, Wireless communication systems and networks.

4. Channel Characterization and Modeling: This chapter discusses the importance of channel characterization for wireless communication systems. It covers frequency selective fading channels, channel models like Raleigh, Nakagami-m, and Weibull, and statistical description of fading channels.

Target Courses: Wireless/mobile communications, Cellular and mobile communications, Wireless communication systems and networks.

5. Modulation Techniques for Wireless Communications: This chapter focuses on various modulation techniques used in wireless communication systems, such as amplitude modulation (AM), frequency modulation (FM), phase shift keying (PSK), and quadrature amplitude modulation (QAM).

Target Courses: Wireless/mobile communications, Cellular and mobile communications, Wireless communication systems and networks.

6. Multiple Access Techniques for Wireless Communications: This chapter covers multiple access techniques used to enable efficient use of wireless communication resources. Topics include frequency division multiple access (FDMA), time division multiple access (TDMA), code division multiple access (CDMA), and orthogonal frequency-division multiplexing (OFDM).

Target Courses: Wireless/mobile communications, Cellular and mobile communications, Wireless communication systems and networks.

7. Cellular Communication Systems: This chapter discusses the basics of cellular communication systems, including their architecture, design, frequency reuse, and capacity enhancement techniques such as sectorization, inter-cell coordination, and antenna tilting.

Target Courses: Wireless/mobile communications, Cellular and mobile communications.

8. Emerging Technologies in Wireless Communications: This chapter introduces emerging technologies in wireless communications, including WiMAX, Long-Term Evolution (LTE), 5G, and beyond. It covers the underlying principles, applications, and potential future developments of these systems.

Target Courses: Wireless/mobile communications, Cellular and mobile communications, Advanced communication systems.

9. Wireless Networking Architectures and Protocols: This chapter discusses various wireless networking architectures, including ad-hoc networks, sensor networks, and mesh networks. It also covers protocols such as IEEE 802.11, Zigbee, and Bluetooth.

Target Courses: Wireless communication systems and networks, Advanced communication systems.

10. Security in Wireless Communications: This chapter focuses on security aspects of wireless communications, including authentication, encryption, and confidentiality. It covers standards like WPA and WPA2, as well as emerging security technologies.

Target Courses: Wireless/mobile communications, Cellular and mobile communications, Wireless communication systems and networks, Advanced communication systems.

11-14: Advanced topics in wireless communications, such as MIMO systems, cognitive radio, satellite communication systems, and optical wireless communication, respectively.

Conclusion:
Wireless Communications is an essential textbook for students and professionals seeking a comprehensive understanding of the field. It covers the fundamental principles of wireless communications, cellular architecture, designs, and emerging networking technologies. The text is modular and can be used to create courses tailored to various target audiences and their specific interests in wireless communication systems.